In [1]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import os
import pandas as pd
import math

def rgbtohsi(rgb_lwpImg):
    rows = int(rgb_lwpImg.shape[0])
    cols = int(rgb_lwpImg.shape[1])
    b, g, r = cv2.split(rgb_lwpImg)
    b = b / 255.0
    g = g / 255.0
    r = r / 255.0
    count=0.0
    for i in range(rows):
        for j in range(cols):
            sum = b[i, j]+g[i, j]+r[i, j]
            I = sum/3.0
            if I*255 >= 100:
                count = count+1
    return count/(rows*cols)


def zmMinFilterGray(src, r=7):
    '''最小值滤波，r是滤波器半径'''
    if r <= 0:
        return src
    h, w = src.shape[:2]
    I = src
    res = np.minimum(I, I[[0] + range(h - 1), :])
    res = np.minimum(res, I[range(1, h) + [h - 1], :])
    I = res
    res = np.minimum(I, I[:, [0] + range(w - 1)])
    res = np.minimum(res, I[:, range(1, w) + [w - 1]])
    return zmMinFilterGray(res, r - 1)


def guidedfilter(I, p, r, eps):
    '''引导滤波'''
    height, width = I.shape
    m_I = cv2.boxFilter(I, -1, (r, r))
    m_p = cv2.boxFilter(p, -1, (r, r))
    m_Ip = cv2.boxFilter(I * p, -1, (r, r))
    cov_Ip = m_Ip - m_I * m_p
    m_II = cv2.boxFilter(I * I, -1, (r, r))
    var_I = m_II - m_I * m_I
    a = cov_Ip / (var_I + eps)
    b = m_p - a * m_I
    m_a = cv2.boxFilter(a, -1, (r, r))
    m_b = cv2.boxFilter(b, -1, (r, r))
    return m_a * I + m_b


def getV1(m, r, eps, w, maxV1):  # 输入rgb图像，值范围[0,1]
    '''计算大气遮罩图像V1和光照值A, V1 = 1-t/A'''
    V1 = np.min(m, 2)  # 得到暗通道图像
    V1 = guidedfilter(V1, zmMinFilterGray(V1, 7), r, eps)  # 使用引导滤波优化
    bins = 2000
    ht = np.histogram(V1, bins)  # 计算大气光照A
    d = np.cumsum(ht[0]) / float(V1.size)
    for lmax in range(bins - 1, 0, -1):
        if d[lmax] <= 0.999:
            break
    A = np.mean(m, 2)[V1 >= ht[1][lmax]].max()

    V1 = np.minimum(V1 * w, maxV1)  # 对值范围进行限制

    return V1, A


def deHaze(m, r=81, eps=0.001, w=0.95, maxV1=0.80, bGamma=False):
    Y = np.zeros(m.shape)
    V1, A = getV1(m, r, eps, w, maxV1)  # 得到遮罩图像和大气光照
    for k in range(3):
        Y[:, :, k] = (m[:, :, k] - V1) / (1 - V1 / A)  # 颜色校正
    Y = np.clip(Y, 0, 1)
    if bGamma:
        Y = Y ** (np.log(0.5) / np.log(Y.mean()))  # gamma校正,默认不进行该操作
    return Y


def custom_blur_demo(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32) #锐化
    dst = cv2.filter2D(image, -1, kernel=kernel)
    return dst

In [ ]:
ship_dir = '/datasets/ee285f-public/airbus_ship_detection/'
train_image_dir = r'/datasets/ee285f-public/airbus_ship_detection/train_v2/' 
for fpathe, dirs, fs in os.walk(train_image_dir):
    for f in fs:
        Filepath = os.path.join(fpathe, f)
        img = cv2.imread(Filepath)
        # cv2.imshow('ori', img)
        blur = cv2.medianBlur(img, 7)#9,21
        # cv2.imshow('blur',blur)
        gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
        # open to see how to use: cv2.Canny
        edges = cv2.Canny(gray, 0, 150, apertureSize=3)#3
        # lines = cv2.HoughLines(edges, 1, np.pi / 180, 25)#25,11
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=1,minLineLength=5,maxLineGap=5)  # 25,11,(15,10)
        if lines is not None:
#             for i in range(len(lines)):
#                     for x1, y1,x2,y2 in lines[i]:
#                         cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255))
            df=pd.DataFrame(data=[f],columns=['maybe'])
            df.to_csv('./maybe.csv', mode='a',index=False, header=False )
        else:
            if rgbtohsi(img) > 0.5:
                img_dehaze = deHaze(img / 255.0) * 255
                img_dehaze = img_dehaze.astype(np.uint8)
                blur = cv2.medianBlur(img_dehaze, 7)  # 9,21
                gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
                edges = cv2.Canny(gray, 0, 150, apertureSize=3)  # 3
                lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=1, minLineLength=5,
                                        maxLineGap=5)  # 25,11,(15,10)
                if lines is not None:
#                     for i in range(len(lines)):
#                             for x1, y1, x2, y2 in lines[i]:
#                                 cv2.line(img_dehaze, (x1, y1), (x2, y2), (0, 0, 255))
                    df=pd.DataFrame(data=[f],columns=['maybe'])
                    df.to_csv('./maybe.csv', mode='a',index=False, header=False )
                else:
                    df=pd.DataFrame(data=[f],columns=['background'])
                    df.to_csv('./bg.csv', mode='a',index=False, header=False )
            else:
                df=pd.DataFrame(data=[f],columns=['background'])
                df.to_csv('./bg.csv', mode='a',index=False, header=False )
    if lines is not None:
        df=pd.DataFrame(data=[f],columns=['maybe'])
        df.to_csv('./maybe.csv', mode='a',index=False, header=False )
    else:
        df=pd.DataFrame(data=[f],columns=['background'])
        df.to_csv('./bg.csv', mode='a',index=False, header=False )
print('finished')

In [2]:
import cv2
import numpy as np
import os
import pandas as pd
ship_dir = '/datasets/ee285f-public/airbus_ship_detection/'
train_image_dir = r'/datasets/ee285f-public/airbus_ship_detection/train_v2/' 
imgs = set()
csv_filename = os.path.join(ship_dir,
                             '/datasets/ee285f-public/airbus_ship_detection/train_ship_segmentations_v2.csv')
csv_data = pd.read_csv(csv_filename)
csv_data_true = csv_data.isnull()
for i in range(csv_data.shape[0]):
    if csv_data_true['EncodedPixels'][i]:
        imgs.add(csv_data['ImageId'].values[i])
bg_data =  pd.read_csv('./bg.csv',header=None)
count_err=0
for i in range(bg_data.shape[0]):
    if bg_data[0][i] not in(imgs):
        count_err=count_err+1
print(count_err*1.0/bg_data.shape[0])

0.040986515259
